In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path

# sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin, clone

# Importações locais
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
from src.preprocess_utils_tic import preprocessador_titanic

## 2. Dataload & Preprocessamento com joblib

In [2]:
BASE = Path.cwd().parent   
# =====================================================
# ⚙️ 0. carregamento dos preprocessador 
# =====================================================
temp = joblib.load(BASE /'src'/'preprocess_Titanic_v1.0.joblib')
PP2=temp['preprocessador']

# # =====================================================
# # 📁 1. Leitura dos dados & Separação das bases
# # =====================================================

DATA_DIR = BASE / "data" / "processed"
X_train = pd.read_csv(DATA_DIR / "X_train_final.csv").reset_index(drop=True)
X_val  = pd.read_csv(DATA_DIR / "X_test_final.csv")
y_train = pd.read_csv(DATA_DIR / "y_train_final.csv").values.ravel()
y_val  = pd.read_csv(DATA_DIR / "y_test_final.csv")


# =====================================================
# 📁 2. Modelo e pipeline
# =====================================================
pipe_RF3 = joblib.load('modelo_RF_final_bayes.joblib')
pipe_RF3.named_steps # informações sobre a pipeline

{'preprocess': preprocessador_titanic(),
 'model': RandomForestClassifier(criterion='entropy', max_depth=8, max_leaf_nodes=52,
                        min_samples_split=15, n_estimators=373, n_jobs=-1,
                        random_state=42)}

In [3]:
# =====================================================
# 3.Treino & Teste
# =====================================================
cv_scores = cross_val_score(pipe_RF3 , X_train, y_train, cv=10)
pipe_RF3.fit(X_train, y_train)
y_pred=pipe_RF3.predict(X_val) #validação
print(f"{'='*70}")
print(f"🎯 Random Forest 2 | cvscores : {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"{'='*70}")
print(f"📊 **Acurácia no Teste**: {accuracy_score(y_val, y_pred):.4f}")
print(f"\n📋 **Relatório de Classificação**:")
print(classification_report(y_val, y_pred))
cm=confusion_matrix(y_val, y_pred)
print(f"🎯 **Matriz de Confusão**:")
print(f"               Previsto 0   Previsto 1")
print(f"Real 0         {cm[0,0]:<11} {cm[0,1]:<11}")
print(f"Real 1         {cm[1,0]:<11} {cm[1,1]:<11}")
print(f"{'─'*70}")

🎯 Random Forest 2 | cvscores : 0.8444 ± 0.0610
📊 **Acurácia no Teste**: 0.8060

📋 **Relatório de Classificação**:
              precision    recall  f1-score   support

           0       0.81      0.88      0.84       157
           1       0.80      0.70      0.75       111

    accuracy                           0.81       268
   macro avg       0.81      0.79      0.80       268
weighted avg       0.81      0.81      0.80       268

🎯 **Matriz de Confusão**:
               Previsto 0   Previsto 1
Real 0         138         19         
Real 1         33          78         
──────────────────────────────────────────────────────────────────────


## 3. Kaggle

In [4]:
# =====================================================
# Submissão Kaggle
# =====================================================
base = pd.read_csv("/home/akel/PycharmProjects/Kaggle/Titanic/data/raw/test.csv")

id_test = base["PassengerId"]

x_test=base.drop(columns='PassengerId')

# Previsão
y_pred=pipe_RF3.predict(x_test)

#  DataFrame de submissão
submission = pd.DataFrame({
    "PassengerId": id_test,
    "Survived": y_pred
})

submission_path = ("/home/akel/PycharmProjects/Kaggle/Titanic/data/processed/submission_RF_bayer_v1.csv")
submission.to_csv(submission_path, index=False)
print("✅ Arquivo de submissão salvo com sucesso!")

✅ Arquivo de submissão salvo com sucesso!
